In [1]:
import json
import os
import glob
from config import problem_configs

# Configuration
SOURCE_DIR = "individual_results"
FINAL_OUTPUT_DIR = "isingBatches"

In [ ]:
#note that this will need updating after i get IONQ working as file names will be different
def merge_all_problem_classes():
    """
    Finds and merges individual result files for each problem class
    defined in the problem_configs dictionary.
    """
    # Loop through each problem type defined in the config
    for problem_name, config in problem_configs.items():
        print(f"--- Processing class: {problem_name} ---")

        file_slug = config['file_slug']
        print(f"File slug for {problem_name}: {file_slug}")

        search_pattern = os.path.join(SOURCE_DIR, f"{file_slug}result_instance_*.json")
        individual_files = glob.glob(search_pattern)

        if not individual_files:
            print(f"No result files found for pattern: {search_pattern}. Skipping.\n")
            continue  # Move to the next problem class
    
        print(f"Found {len(individual_files)} result files to merge.")

        all_results = []
        metadata = {}

        # Loop through each found file, load its data, and append the result
        for file_path in individual_files:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            all_results.append(data["result"])
            
            if not metadata:
                metadata = data["metadata"]

        all_results.sort(key=lambda x: x['instance_id'])

        # Assemble the final data structure for this problem class
        final_data = {
            "metadata": metadata,
            "results": all_results
        }

        # 2. Dynamically create the final output filename
        final_output_filename = f"MERGED_QAOA_results_{file_slug}.json"
        final_path = os.path.join(FINAL_OUTPUT_DIR, final_output_filename)

        os.makedirs(FINAL_OUTPUT_DIR, exist_ok=True)

        with open(final_path, 'w') as f:
            json.dump(final_data, f, indent=4)

        print(f"Successfully merged {len(all_results)} results into {final_path}\n")


if __name__ == "__main__":
    merge_all_problem_classes()

--- Processing class: TSP ---
File slug for TSP: TSP_9q_
Found 100 result files to merge.
Successfully merged 100 results into isingBatches/MERGED_QAOA_results_TSP_9q_.json

--- Processing class: Knapsack ---
File slug for Knapsack: Knapsack_6_items_9q_
Found 100 result files to merge.
Successfully merged 100 results into isingBatches/MERGED_QAOA_results_Knapsack_6_items_9q_.json

--- Processing class: MinimumVertexCover ---
File slug for MinimumVertexCover: MinimumVertexCover_9q_
Found 100 result files to merge.
Successfully merged 100 results into isingBatches/MERGED_QAOA_results_MinimumVertexCover_9q_.json

